In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 회원정보

In [3]:
df1 = pd.read_parquet('../open/train/1.회원정보/201807_train_회원정보.parquet')
df2 = pd.read_parquet('../open/train/1.회원정보/201808_train_회원정보.parquet')
df3 = pd.read_parquet('../open/train/1.회원정보/201809_train_회원정보.parquet')
df4 = pd.read_parquet('../open/train/1.회원정보/201810_train_회원정보.parquet')
df5 = pd.read_parquet('../open/train/1.회원정보/201811_train_회원정보.parquet')
df6 = pd.read_parquet('../open/train/1.회원정보/201812_train_회원정보.parquet')

In [4]:
# 담을 리스트 리스트
all_df = []

# df1~6 담아주기
all_df.append(df1)
all_df.append(df2)
all_df.append(df3)
all_df.append(df4)
all_df.append(df5)
all_df.append(df6)

# merge
merged_df = pd.concat(all_df, ignore_index=True)

print("\n--- 데이터 기본 정보 ---")
merged_df.info()
print("\n")
merged_df.head()


--- 데이터 기본 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 78 columns):
 #   Column              Dtype  
---  ------              -----  
 0   기준년월                int64  
 1   ID                  object 
 2   남녀구분코드              int64  
 3   연령                  object 
 4   Segment             object 
 5   회원여부_이용가능           int64  
 6   회원여부_이용가능_CA        int64  
 7   회원여부_이용가능_카드론       int64  
 8   소지여부_신용             int64  
 9   소지카드수_유효_신용         int64  
 10  소지카드수_이용가능_신용       int64  
 11  입회일자_신용             int64  
 12  입회경과개월수_신용          int64  
 13  회원여부_연체             int64  
 14  이용거절여부_카드론          int64  
 15  동의여부_한도증액안내         int64  
 16  수신거부여부_TM           int64  
 17  수신거부여부_DM           int64  
 18  수신거부여부_메일           int64  
 19  수신거부여부_SMS          int64  
 20  가입통신회사코드            object 
 21  탈회횟수_누적             int64  
 22  최종탈회후경과월            int64  
 23  탈회횟수_발급6개월이내        int64  
 24  탈회횟수_

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,...,할인금액_제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,상품관련면제카드수_B0M,임직원면제카드수_B0M,우수회원면제카드수_B0M,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),22
1,201807,TRAIN_000001,1,30대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),18
2,201807,TRAIN_000002,1,30대,C,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀출산기,20
3,201807,TRAIN_000003,2,40대,D,1,1,0,1,2,...,0,0,0,0개,0개,0개,0개,1,자녀성장(2),17
4,201807,TRAIN_000004,2,40대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,1,자녀성장(1),15


In [5]:
print("회원정보 기본정보")
print("--- 결측치 ---")
missing_data = merged_df.isnull().sum()
missing_percent = 100 * merged_df.isnull().sum() / len(merged_df)
missing_df = pd.DataFrame({'결측치 수': missing_data, '비율': missing_percent})
missing_df = missing_df[missing_df['결측치 수'] > 0].sort_values(by='비율', ascending=False)
print(missing_df)

# 결측치_많은컬럼 50% 이상
# 논의 후 재조정
high_missing = missing_df[missing_df['비율'] > 50].index.tolist()
if high_missing:
    print(f"\n높은 결측치: {high_missing}")

# 고유값 개수 확인
single_value_cols = []
print("\n--- 고정된 값 확인 ---")
for col in merged_df.columns:
    unique_value_count = merged_df[col].nunique(dropna=False)

    # 고유한 값의 개수가 1개인 경우
    if unique_value_count == 1:
        single_value_cols.append(col)

print(f"\n고정값 : \n{single_value_cols}")


# 컬럼 값이 완전히 동일한 중복 컬럼 찾기
# 병합 후 재실행
print("\n--- 중복 컬럼 확인 ---")
duplicated_cols = {}
cols = merged_df.columns.tolist()
for i in range(len(cols)):
    for j in range(i + 1, len(cols)):
        col1 = cols[i]
        col2 = cols[j]
        if merged_df[col1].equals(merged_df[col2]):
            if col1 not in duplicated_cols:
                duplicated_cols[col1] = []
            duplicated_cols[col1].append(col2)
if duplicated_cols:
    print(f"중복 컬럼들: \n{duplicated_cols}")

# 분산 확인
print("\n--- 분산 확인 ---")
n1 = merged_df.select_dtypes(include=['int64', 'float64'])
variance1 = n1.var()
low_variance_cols = variance1[variance1 < 0.1].index.tolist()
print(f"\n낮은 분산 : \n{low_variance_cols}")

# 0이 대부분인 컬럼
print("\n--- 0컬럼 확인 ---")
zero_per_limit = 0.7
cols_to_remove = []

numerical_cols = merged_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

for col in numerical_cols:
    zero_count = (merged_df[col] == 0).sum()

    # 0의 비율
    total_count_col = len(merged_df[col]) 
    zero_percentage = zero_count / total_count_col

    # 0이 아닌 유효한 값의 개수
    count1 = merged_df[col].count() - zero_count

    if zero_percentage >= zero_per_limit and count1 < 1000000:
        cols_to_remove.append(col)
        print(f"'{col}': 0의 비율 = {zero_percentage:.2%}, 0이 아닌 유효값 수 = {count1}")

print(f"\n대부분 0인 컬럼:\n {cols_to_remove}")

회원정보 기본정보
--- 결측치 ---
                 결측치 수         비율
_2순위신용체크구분      958115  39.921458
최종유효년월_신용_이용    534231  22.259625
가입통신회사코드        387570  16.148750
직장시도명           244969  10.207042
최종유효년월_신용_이용가능  210447   8.768625
최종카드발급일자         41965   1.748542
_1순위신용체크구분       27950   1.164583

--- 고정된 값 확인 ---

고정값 : 
['이용카드수_체크_가족', '이용금액_R3M_체크_가족', '연회비할인카드수_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M']

--- 중복 컬럼 확인 ---
중복 컬럼들: 
{'이용카드수_체크_가족': ['이용금액_R3M_체크_가족', '연회비할인카드수_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M'], '이용금액_R3M_체크_가족': ['연회비할인카드수_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M'], '연회비할인카드수_B0M': ['할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M'], '기본연회비_B0M': ['청구금액_기본연회비_B0M'], '제휴연회비_B0M': ['청구금액_제휴연회비_B0M'], '할인금액_기본연회비_B0M': ['할인금액_제휴연회비_B0M'], '상품관련면제카드수_B0M': ['임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M'], '임직원면제카드수_B0M': ['우수회원면제카드수_B0M', '기타면제카드수_B0M'], '우수회원면제카드수_B0M': ['기타면제카드수_B0M']}

--- 분산 확인 ---

낮은 분산 : 
['회원여부_이

In [6]:
print(merged_df['이용카드수_체크_가족'].unique())
print(merged_df['이용금액_R3M_체크_가족'].unique())
print(merged_df['연회비할인카드수_B0M'].unique())
print(merged_df['기본연회비_B0M'].unique())
print(merged_df['제휴연회비_B0M'].unique())
print(merged_df['할인금액_기본연회비_B0M'].unique())
print(merged_df['상품관련면제카드수_B0M'].unique())
print(merged_df['임직원면제카드수_B0M'].unique())
print(merged_df['우수회원면제카드수_B0M'].unique())

# '기본연회비_B0M'와 '제휴 연회비_B0M'는 '정구금액_ 기본연회비_B0M'와 '청구금액_제휴 연회비_B0M' 와 동일

[0]
[0]
[0]
[  0 106 109 108 105  50 107 103 104 110  49 111 102 101]
[  0 950 949  30 149  29 951 150]
[0]
['0개']
['0개']
['0개']


In [13]:
flattened_duplicated_cols = [item for sublist in duplicated_cols.values() for item in sublist]
all_columns_to_drop_set = set(high_missing +
                              single_value_cols +
                              flattened_duplicated_cols +
                              low_variance_cols +
                              cols_to_remove)

# 다시 리스트로 변환
columns_to_drop1 = list(all_columns_to_drop_set)

In [15]:
print(f"제거 전 컬럼의 수 : {len(merged_df.columns)}")
print(f"제거할 컬럼의 수 : {len(columns_to_drop1)}")
print(f"제거 후 컬럼의 수 : {len(merged_df.columns)-len(columns_to_drop1)}")

제거 전 컬럼의 수 : 78
제거할 컬럼의 수 : 31
제거 후 컬럼의 수 : 47


In [ ]:
10/0

# 신용정보

In [ ]:
df1 = pd.read_parquet('../open/train/2.신용정보/201807_train_신용정보.parquet')
df2 = pd.read_parquet('../open/train/2.신용정보/201808_train_신용정보.parquet')
df3 = pd.read_parquet('../open/train/2.신용정보/201809_train_신용정보.parquet')
df4 = pd.read_parquet('../open/train/2.신용정보/201810_train_신용정보.parquet')
df5 = pd.read_parquet('../open/train/2.신용정보/201811_train_신용정보.parquet')
df6 = pd.read_parquet('../open/train/2.신용정보/201812_train_신용정보.parquet')

In [ ]:
# 담을 리스트 리스트
all_df = []

# df1~6 담아주기
all_df.append(df1)
all_df.append(df2)
all_df.append(df3)
all_df.append(df4)
all_df.append(df5)
all_df.append(df6)

# merge
merged_df = pd.concat(all_df, ignore_index=True)

print("\n--- 데이터 기본 정보 ---")
merged_df.info()
print("\n")
merged_df.head()

In [ ]:
print("신용정보 기본정보")
print("--- 결측치 ---")
missing_data = merged_df.isnull().sum()
missing_percent = 100 * merged_df.isnull().sum() / len(merged_df)
missing_df = pd.DataFrame({'결측치 수': missing_data, '비율': missing_percent})
missing_df = missing_df[missing_df['결측치 수'] > 0].sort_values(by='비율', ascending=False)
print(missing_df)

# 결측치_많은컬럼 50% 이상
# 논의 후 재조정
high_missing = missing_df[missing_df['비율'] > 50].index.tolist()
if high_missing:
    print(f"\n높은 결측치: {high_missing}")

# 고유값 개수 확인
single_value_cols = []
print("\n--- 고정된 값 확인 ---")
for col in merged_df.columns:
    unique_value_count = merged_df[col].nunique(dropna=False)

    # 고유한 값의 개수가 1개인 경우
    if unique_value_count == 1:
        single_value_cols.append(col)

print(f"\n고정값 : \n{single_value_cols}")


# 컬럼 값이 완전히 동일한 중복 컬럼 찾기
# 병합 후 재실행
print("\n--- 중복 컬럼 확인 ---")
duplicated_cols = {}
cols = merged_df.columns.tolist()
for i in range(len(cols)):
    for j in range(i + 1, len(cols)):
        col1 = cols[i]
        col2 = cols[j]
        if merged_df[col1].equals(merged_df[col2]):
            if col1 not in duplicated_cols:
                duplicated_cols[col1] = []
            duplicated_cols[col1].append(col2)
if duplicated_cols:
    print(f"중복 컬럼들: \n{duplicated_cols}")

# 분산 확인
print("\n--- 분산 확인 ---")
n1 = merged_df.select_dtypes(include=['int64', 'float64'])
variance1 = n1.var()
low_variance_cols = variance1[variance1 < 0.1].index.tolist()
print(f"\n낮은 분산 : \n{low_variance_cols}")

# 0이 대부분인 컬럼
print("\n--- 0컬럼 확인 ---")
zero_per_limit = 0.7
cols_to_remove = []

numerical_cols = merged_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

for col in numerical_cols:
    zero_count = (merged_df[col] == 0).sum()

    # 0의 비율
    total_count_col = len(merged_df[col]) 
    zero_percentage = zero_count / total_count_col

    # 0이 아닌 유효한 값의 개수
    count1 = merged_df[col].count() - zero_count

    if zero_percentage >= zero_per_limit and count1 < 1000000:
        cols_to_remove.append(col)
        print(f"'{col}': 0의 비율 = {zero_percentage:.2%}, 0이 아닌 유효값 수 = {count1}")

print(f"\n대부분 0인 컬럼:\n {cols_to_remove}")

In [ ]:
flattened_duplicated_cols = [item for sublist in duplicated_cols.values() for item in sublist]
all_columns_to_drop_set = set(high_missing +
                              single_value_cols +
                              flattened_duplicated_cols +
                              low_variance_cols +
                              cols_to_remove)

# 다시 리스트로 변환
columns_to_drop2 = list(all_columns_to_drop_set)

In [ ]:
print(f"제거 전 컬럼의 수 : {len(merged_df.columns)}")
print(f"제거할 컬럼의 수 : {len(columns_to_drop2)}")
print(f"제거 후 컬럼의 수 : {len(merged_df.columns)-len(columns_to_drop2)}")